In [1]:
# 01_import_und_setup — Cell 1
from __future__ import annotations

from pathlib import Path
import sys, os, json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import re
from datetime import datetime

# ---------- Projektpfade robust ermitteln ----------
# Falls Notebook in /notebooks läuft: gehe eine Ebene hoch (= Projektwurzel)
NB_DIR  = Path.cwd().resolve()
BASE    = NB_DIR.parents[0] if NB_DIR.name.lower() == "notebooks" else NB_DIR
DATA    = BASE / "data"
RAW     = DATA / "raw"
OUT     = DATA / "processed"
REPORTS = BASE / "reports"
FIG     = REPORTS / "figures"

for p in (RAW, OUT, FIG):
    p.mkdir(parents=True, exist_ok=True)

# ---------- Pandas & Matplotlib-Grundsetup ----------
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

def apply_scientific_style():
    plt.rcParams.update({
        "figure.dpi": 120,
        "savefig.dpi": 300,
        "font.size": 11,
        "axes.titlesize": 12,
        "axes.labelsize": 11,
        "axes.grid": True,
        "grid.alpha": 0.2,
        "axes.spines.top": False,
        "axes.spines.right": False,
        "axes.formatter.use_locale": False,
    })

apply_scientific_style()

def percent_axis(ax, axis="y", decimals=0, limit=(0, 100)):
    """Einheitliche Prozent-Achse (0–100 %) für wissenschaftlich korrekte Plots."""
    fmt = mtick.PercentFormatter(xmax=100, decimals=decimals)
    if axis == "y":
        ax.set_ylim(*limit)
        ax.yaxis.set_major_formatter(fmt)
    else:
        ax.set_xlim(*limit)
        ax.xaxis.set_major_formatter(fmt)


In [2]:
# 01_import_und_setup — Cell 2
# ---------- Ziel-Kategorien (harmonisiert) ----------
KANON: list[str] = [
    "Kleidung / Schuhe",
    "Elektronik (z. B. Smartphones, Haushaltsgeräte)",
    "Lebensmittel / Getränke",
    "Bücher / Medien / Software",
    "Medikamente / Drogerieartikel",
    "Hobby- & Freizeitartikel",
    "Möbel / Wohnaccessoires",
]

# ---------- Quell-Mapping (Statista-Begriffe → KANON) ----------
MAP_STATISTA: dict[str, str] = {
    # 2021
    "Fashion & Accessoires": "Kleidung / Schuhe",
    "Freizeit": "Hobby- & Freizeitartikel",
    "Kosmetik & Drogerie": "Medikamente / Drogerieartikel",
    "Medikamente & Gesundheit": "Medikamente / Drogerieartikel",
    "Wohnen & Einrichten": "Möbel / Wohnaccessoires",
    "Consumer Electronics/Elektrogeräte": "Elektronik (z. B. Smartphones, Haushaltsgeräte)",
    "Heimwerken & Garten": "Hobby- & Freizeitartikel",
    "Sport- und Outdoorausrüstung": "Hobby- & Freizeitartikel",
    "Lebensmittel": "Lebensmittel / Getränke",
    "Getränke": "Lebensmittel / Getränke",
    # 2022
    "Kleidung, Schuhe, Accessoires": "Kleidung / Schuhe",
    "Bücher, Hörbücher": "Bücher / Medien / Software",
    "Elektrozubehör": "Elektronik (z. B. Smartphones, Haushaltsgeräte)",
    "Medikamente": "Medikamente / Drogerieartikel",
    "Kosmetik, Parfum und Pflegeprodukte": "Medikamente / Drogerieartikel",
    "Möbel, Wohnen und Dekoration": "Möbel / Wohnaccessoires",
    "Elektronische Haushaltsgeräte": "Elektronik (z. B. Smartphones, Haushaltsgeräte)",
    "Lebensmittel, Getränke": "Lebensmittel / Getränke",
    "Heimwerkerbedarf": "Hobby- & Freizeitartikel",
    "Spielzeug / Spiele": "Hobby- & Freizeitartikel",
    "Smartphones / Handys": "Elektronik (z. B. Smartphones, Haushaltsgeräte)",
    "Musik / Filme auf CD / DVD / Blu-ray": "Bücher / Medien / Software",
    "Computer: Desktop-PC, Tablet, Laptop": "Elektronik (z. B. Smartphones, Haushaltsgeräte)",
    "Gartengeräte": "Hobby- & Freizeitartikel",
    "Software": "Bücher / Medien / Software",
    "Unterhaltungselektronik": "Elektronik (z. B. Smartphones, Haushaltsgeräte)",
    # 2024
    "Bekleidung": "Kleidung / Schuhe",
    "Schuhe": "Kleidung / Schuhe",
    "Accessoires": "Kleidung / Schuhe",
    "Drogerie & Gesundheit": "Medikamente / Drogerieartikel",
    "Kosmetik & Körperpflege": "Medikamente / Drogerieartikel",
    "Bücher, Filme, Musik & Spiele (keine Downloads)": "Bücher / Medien / Software",
    "Unterhaltungselektronik (z. B. Fernseher, Smartphones)": "Elektronik (z. B. Smartphones, Haushaltsgeräte)",
    "Haushaltsgeräte": "Elektronik (z. B. Smartphones, Haushaltsgeräte)",
    "Lebensmittel und Getränke (ausgenommen Lieferungen von Restaurants)": "Lebensmittel / Getränke",
    "Möbel & Haushaltswaren": "Möbel / Wohnaccessoires",
    "Spielzeug & Babyprodukte": "Hobby- & Freizeitartikel",
    "Sport- & Outdoor-Artikel": "Hobby- & Freizeitartikel",
    "DIY, Heimwerker- und Gartenbedarf": "Hobby- & Freizeitartikel",
    "Hobby & Schreibwaren": "Hobby- & Freizeitartikel",
]


SyntaxError: closing parenthesis ']' does not match opening parenthesis '{' on line 14 (4148564128.py, line 58)

In [ ]:
# 01_import_und_setup — Cell 3
def norm_text(s: str) -> str:
    """Robuste Text-Normalisierung für Mapping-Lookups."""
    if pd.isna(s):
        return ""
    s = str(s).strip().lower()
    s = s.replace("&", "und").replace("-", " ")
    s = re.sub(r"\s+", " ", s)
    # häufige Tippfehler/Varianten
    s = s.replace("accressoires", "accessoires").replace("hi tech", "high tech")
    return s

def make_map_norm(map_statista: dict[str, str]) -> dict[str, str]:
    return {norm_text(k): v for k, v in map_statista.items()}

MAP_NORM: dict[str, str] = make_map_norm(MAP_STATISTA)

# Sanity-Check: doppelte Normalisate?
from collections import Counter
_counts = Counter(norm_text(k) for k in MAP_STATISTA)
_dups = [k for k, n in _counts.items() if n > 1]
if _dups:
    print("Warnung: doppelte normalisierte Keys im Mapping:", _dups)
else:
    print("Mapping OK – keine doppelten normalisierten Keys.")


In [ ]:
# 01_import_und_setup — Cell 4
# Persistentes Mapping (für Audits & spätere Anpassungen ohne Codeänderung)
mapping_df = pd.DataFrame(
    [{"source": k, "source_normalized": norm_text(k), "kanon": v} for k, v in MAP_STATISTA.items()]
).sort_values(["kanon", "source"]).reset_index(drop=True)

MAP_CSV = OUT / "mapping_statista_to_kanon.csv"
mapping_df.to_csv(MAP_CSV, index=False, encoding="utf-8")
display(mapping_df.head(10))

# Leichtgewichtige Projektkonfiguration (Artefakt-Pfade etc.)
CONFIG = {
    "generated_at": datetime.now().isoformat(timespec="seconds"),
    "base": str(BASE),
    "paths": {
        "raw": str(RAW),
        "processed": str(OUT),
        "reports": str(REPORTS),
        "figures": str(FIG),
        "mapping_csv": str(MAP_CSV),
    },
    "kanon": KANON,
}
(CONFIG_PATH := OUT / "project_config.json").write_text(json.dumps(CONFIG, indent=2, ensure_ascii=False))
print("Konfiguration gespeichert →", CONFIG_PATH)


In [ ]:
# 01_import_und_setup — Cell 5
def env_summary():
    import platform
    print("== Environment ==")
    print("Python   :", platform.python_version())
    print("Pandas   :", pd.__version__)
    print("Matplotlib:", plt.matplotlib.__version__)
    print("Base     :", BASE)
    print("RAW      :", RAW)
    print("OUT      :", OUT)
    print("FIG      :", FIG)
    print("Mapping  :", OUT / "mapping_statista_to_kanon.csv")
    print("Notebook :", NB_DIR)
    print()

env_summary()
print("Setup abgeschlossen. Weiter mit 10_ingest.ipynb → Rohdaten laden & prüfen.")
